<a href="https://colab.research.google.com/github/ordevoir/Publish/blob/master/Elementary_Cellular_Automaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Основные функции

In [ ]:
def int_to_bits(n: int) -> np.ndarray:
    """Функция принимает целое число в диапазоне [0, 255].
    Возвращает массив из 8 элементов – двоичное представление числа.
    Младший бит – крайний левый (с индексом 0)"""
    shifts = np.arange(8, dtype=np.uint64)
    return (n >> shifts) & 1

int_to_bits(42)

In [ ]:
def pass_tape_vec(tape: np.ndarray, lut: np.ndarray, iteration: int) -> None:
    """Функция принимает:
    - матрицу tape – временно́й роллаут одномерного автомата;
    - массив lut – двоичное представление правила переходов (младший бит по индексу 0);
    - положительное целое число iteration – номер текущей итерации;
    Функция заполняет ряд itaration+1 на основе ряда iteration по правилу lut.
    """
    row = tape[iteration]
    L = row[:-2]  << 2      # left      старший бит     умножение на 2**2
    C = row[1:-1] << 1      # center                    умножение на 2**1
    R = row[2:] # << 0      # right     младший бит
    lut_codes = L | C | R   # целочисленные коды триплетов (0..7) – сложение

    tape[iteration+1, 1:-1] = lut[lut_codes]   # обновляем внутренние клетки

# Конфигурирование

In [ ]:
length = 25001               # длина клеточного автомата
rule = 30                 # правила переходов

n_iterations = length // 2

# определим, в каких позициях будут заданы единичные ячейки автомата
init = np.array([
    length // 2,
    # 2* length // 3 - 100,
    ])

## Случайная конфигурация

In [ ]:
# random_values = np.random.randint(0, 2, size=length)
# indices = [i for i in range(len(random_values)) if random_values[i]]
# init = np.array(indices)

# Запуск симуляции

In [ ]:
tape = np.zeros((n_iterations+1, length), dtype=np.uint8)
tape[0, init] = 1

lut = int_to_bits(rule)
for i in range(n_iterations):
    pass_tape_vec(tape, lut, i)

image = tape.astype(np.uint8)
image[image==0] = 20
image[image==1] = 130

# Визуализация

## Визуализация всего роллаута

In [ ]:
# plt.style.use("ordevoir-dark")
plt.figure(figsize=(40, 20))
plt.axis('off')
plt.imshow(image, cmap='gray', vmin=0, vmax=255)

## Визуализация фрагмента

In [ ]:
left = length // 2 - 1000
right = length // 2 + 1000
top = length // 2 - 2000
bottom = length // 2
print(top, bottom, left, right)
# plt.style.use("ordevoir-dark")
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(image[top:bottom, left:right], cmap='gray', vmin=0, vmax=255)

# Сохранение изображения

In [ ]:
import uuid
from PIL import Image


suffix = uuid.uuid4().hex[:8]

# img = Image.fromarray(image)
img = Image.fromarray(image[top:bottom, left:right])    # fragment

img.save(f"./images/rule_{rule}_{suffix}.png")